# SQL

## Описание проекта

Компания решила купила крупный сервис для чтения книг по подписке. Первая задача — проанализировать базу данных PostgresSQL. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задачи проекта:

1. Подключиться к базе данных data-analyst-final-project-db.
2. Проверка данных на соответствие ERD диаграмме.
3. Исследовательский анализ данных.
4. Интерпретация результатов и подготовка выводов.

### Техническое задание

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц и исключить из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг, необходимо учитывать только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Подключение к базе данных

In [1]:
# Импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# Изменение кол-ва символов в ячейке
pd.options.display.max_colwidth = 120

In [2]:
# Устанавливаем параметры подключения к серверу SQL
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# Cохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Получаем список всех таблиц для пользователя praktikum_student
query = '''select * from pg_tables where schemaname='public';'''
con=engine.connect()
tables = pd.io.sql.read_sql(sql=text(query), con = con)
tables = tables[tables.hastriggers == True]
display(tables)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
3,public,authors,praktikum_admin,None,True,False,True,False
6,public,publishers,praktikum_admin,None,True,False,True,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False
10,public,books,praktikum_admin,None,True,False,True,False


## Проверка данных на соответствие ERD диаграмме

<b>ERD cхема базы данных</b></br>
![](https://pictures.s3.yandex.net/resources/scheme_1589269096.png) 

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [4]:
# Обзор данных в таблицах
for tab in tables.tablename:
    try:
        query = '''SELECT * FROM %s''' %tab
        globals()[tab] = pd.io.sql.read_sql(sql=text(query), con = con)
        print(f'Загружены данные в датасет: {tab}')
        print('________________________________________')
        display(locals()[tab].head())
        display(locals()[tab].info())
    except:
        print('Error in upload data')

Загружены данные в датасет: authors
________________________________________


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

Загружены данные в датасет: publishers
________________________________________


...

None

Загружены данные в датасет: books
________________________________________


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

У пользователя praktikum_student есть доступ к базе данных data-analyst-final-project-db, в которой присутствуют следующие таблицы:
1. authors, 636 строк;
2. publishers, 340 строк;
3. reviews, 2793 строк;
4. ratings, 6456 строк;
5. books, 1000 срок.

## Исследовательский анализ данных

### Анализ объема выпуска книг

In [5]:
# Cколько книг вышло после 1 января 2000 года
query = '''
        SELECT COUNT(*)
        FROM books AS bk
        WHERE bk.publication_date > '2000-01-01'
'''
books_date = pd.io.sql.read_sql(sql=text(query), con = con)
print('Количество книг выпущенно с 01 января 2000 года:', books_date['count'].item())


Количество книг выпущенно с 01 января 2000 года: 819


C 01 января 2000 года было издано 821 книги.

### Количество обзоров и средняя оценка

In [6]:
# Для каждой книги посчитаем количество обзоров и среднюю оценку
query = '''
        WITH
                rw AS (
                        SELECT book_id,
                        COUNT(review_id) as count_review
                        FROM reviews
                        GROUP BY book_id
                ),
                rt AS (
                        SELECT book_id,
                        AVG(rating) as avg_rating
                        FROM ratings
                        GROUP BY  book_id
        )
        SELECT bk.title,
               rw.count_review,
               ROUND(rt.avg_rating, 2)
        FROM books AS bk
        LEFT JOIN rw ON bk.book_id = rw.book_id
        LEFT JOIN rt ON bk.book_id = rt.book_id

'''
books_reviews_ratings = pd.io.sql.read_sql(sql=text(query), con = con)
display(books_reviews_ratings)

,title,count_review,round
0,The Body in the Library (Miss Marple #3),2.0,4.50
1,Galápagos,2.0,4.50
2,A Tree Grows in Brooklyn,5.0,4.25
3,Undaunted Courage: The Pioneering First Mission to Explore America's Wild Frontier,2.0,4.00
4,The Prophet,4.0,4.29
...,...,...,...
995,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
996,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.67
997,Essential Tales and Poems,NaN,4.00
998,Leonardo's Notebooks,NaN,4.00


In [7]:
books_reviews_ratings[books_reviews_ratings.count_review.isna()]

,title,count_review,round
994,Disney's Beauty and the Beast (A Little Golden Book),NaN,4.00
995,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
996,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.67
997,Essential Tales and Poems,NaN,4.00
998,Leonardo's Notebooks,NaN,4.00
999,The Natural Way to Draw,NaN,3.00


Посчитали средние оценки и количество просмотров книг.

У 6 книг отсутствуют обзоры, но при этом выставлены рейтинги.  

### Топовое издательство

In [8]:
# Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, исключив из анализа брошюры
query = '''
        SELECT pb.publisher,
               COUNT(bk.title) AS books
        FROM books AS bk
        LEFT JOIN publishers AS pb ON bk.publisher_id = pb.publisher_id
        WHERE bk.num_pages >= 50
        GROUP BY pb.publisher
        ORDER BY books DESC
        LIMIT 1
'''
display(pd.io.sql.read_sql(sql=text(query), con = con))


,publisher,books
0,Penguin Books,42


Издатель Penguin Books выпустил 42 книги, с количеством страниц больше 50.

### Топовый автор

In [9]:
# Определение автора с самой высокой средней оценкой книг, исключаем книги с 50 и более оценками
query = '''
        WITH
        au_rt AS (
                SELECT au.author,
                       AVG(rt.rating) as avg_rating
                FROM books as bk
                LEFT JOIN authors AS au ON bk.author_id = au.author_id
                LEFT JOIN ratings AS rt ON bk.book_id = rt.book_id
                GROUP BY bk.book_id,
                         au.author_id
                HAVING COUNT(rt.rating_id) >=50
        )
        SELECT au_rt.author,
               ROUND(AVG(au_rt.avg_rating),2)
        FROM au_rt
        GROUP BY au_rt.author
        ORDER BY AVG(au_rt.avg_rating) DESC
        LIMIT 1
'''
display(pd.io.sql.read_sql(sql=text(query), con = con))

,author,round
0,J.K. Rowling/Mary GrandPré,4.28


Произведения авторов J.K. Rowling/Mary GrandPré имеют самый высокий средний рейтинг 4.28.

### Среднее количество пользовательских обзоров

In [10]:
# Посчитаем среднее количество обзоров от пользователей, которым поставили больше 48 оценок
query = '''
WITH
rev AS (SELECT username,
             COUNT(review_id) AS review_count
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating_id) > 48)
      GROUP BY username)
SELECT AVG(rev.review_count) AS avg
FROM rev
'''
avg_review = pd.io.sql.read_sql(sql=text(query), con = con)
print('Cреднее количество обзоров от пользователей:', avg_review.avg.item())

Cреднее количество обзоров от пользователей: 24.0


24 - среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

## Итоги

Результаты анализа данных в базе данных data-analyst-final-project-db:

1. После 1 января 2000 года вышло 819 книг.
2. У 6 книг в базе отсутствуют отзывы.
3. Издатель Penguin Books выпустил наибольшее количество книг - 42, с количеством страниц больше 50.
4. Произведения авторов J.K. Rowling/Mary GrandPré имеют самый высокий средний рейтинг - 4.28
5. Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24.